##  最大化 $xy$ 


满足 $x + y^2 ≤ 2$ 

$x, y ≥ 0$

In [1]:
import numpy as np
import ipyvolume.pylab as plt3d

In [2]:
f = lambda X: X[0] * X[1]
h1 = lambda X: 2 - X[0] - X[1]**2
h2 = lambda X: X[0]
h3 = lambda X: X[1]

In [3]:
xn = np.linspace(-5, 5, 50)
yn = np.linspace(-5, 5, 50)
xxn, yyn = np.meshgrid(xn, yn)
fn = f([xxn, yyn])
h1n = h1([xxn, yyn])
h2n = h2([xxn, yyn])
h3n = h3([xxn, yyn])

plt3d.figure(width=1000, height=600)
plt3d.plot_surface(xxn, fn, yyn, color='gray')
plt3d.plot_wireframe(xxn, h1n, yyn, color='green')
plt3d.plot_wireframe(xxn, h2n, yyn, color='blue')
plt3d.plot_wireframe(xxn, h3n, yyn, color='orange')
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

## KKT 条件

In [4]:
import sympy
sympy.init_printing()

In [5]:
x, y = sympy.symbols("x, y")
f_sym = -x * y
h1_sym = 2 - x - y**2
h2_sym = x
h3_sym = y

### 1. 平稳性

In [6]:
f_grad = [f_sym.diff(i) for i in (x, y)]
f_grad = sympy.Matrix(f_grad)
H = sympy.Matrix([h1_sym, h2_sym, h3_sym])
H_grad = [H.diff(i) for i in (x, y)]
H_grad = sympy.Matrix([H_grad[0], H_grad[1]]).reshape(2, 3)
mu1, mu2, mu3 = sympy.symbols("mu1, mu2, mu3")
mu = sympy.Matrix([mu1, mu2, mu3])

stat_cond = f_grad - H_grad * mu
stat_cond

⎡  μ₁ - μ₂ - y  ⎤
⎢               ⎥
⎣2⋅μ₁⋅y - μ₃ - x⎦

### 2. 原始可行性

In [7]:
prim_h1 = sympy.simplify(h1_sym >= 0)
prim_h2 = sympy.simplify(x >= 0)
prim_h3 = sympy.simplify(y >= 0)
prim_cond = sympy.Matrix([prim_h1, prim_h2, prim_h3])
prim_cond

⎡     2        ⎤
⎢x + y  - 2 ≤ 0⎥
⎢              ⎥
⎢    x ≥ 0     ⎥
⎢              ⎥
⎣    y ≥ 0     ⎦

### 3. 互补松弛性

In [8]:
slack_cond = mu.multiply_elementwise(H)
slack_cond

⎡   ⎛      2    ⎞⎤
⎢μ₁⋅⎝-x - y  + 2⎠⎥
⎢                ⎥
⎢      μ₂⋅x      ⎥
⎢                ⎥
⎣      μ₃⋅y      ⎦

### 4. 对偶可行性

In [9]:
dual_mu = sympy.Matrix([mu1>=0, mu2>=0, mu3>=0])
dual_mu

⎡μ₁ ≥ 0⎤
⎢      ⎥
⎢μ₂ ≥ 0⎥
⎢      ⎥
⎣μ₃ ≥ 0⎦

### 组成最优性

In [10]:
optima_cond = sympy.Matrix([slack_cond, stat_cond, prim_cond, dual_mu])
optima_cond

⎡   ⎛      2    ⎞⎤
⎢μ₁⋅⎝-x - y  + 2⎠⎥
⎢                ⎥
⎢      μ₂⋅x      ⎥
⎢                ⎥
⎢      μ₃⋅y      ⎥
⎢                ⎥
⎢  μ₁ - μ₂ - y   ⎥
⎢                ⎥
⎢2⋅μ₁⋅y - μ₃ - x ⎥
⎢                ⎥
⎢      2         ⎥
⎢ x + y  - 2 ≤ 0 ⎥
⎢                ⎥
⎢     x ≥ 0      ⎥
⎢                ⎥
⎢     y ≥ 0      ⎥
⎢                ⎥
⎢     μ₁ ≥ 0     ⎥
⎢                ⎥
⎢     μ₂ ≥ 0     ⎥
⎢                ⎥
⎣     μ₃ ≥ 0     ⎦

### 求解 

In [11]:
optima_cond = sympy.Matrix([slack_cond, stat_cond])
optima_cond

⎡   ⎛      2    ⎞⎤
⎢μ₁⋅⎝-x - y  + 2⎠⎥
⎢                ⎥
⎢      μ₂⋅x      ⎥
⎢                ⎥
⎢      μ₃⋅y      ⎥
⎢                ⎥
⎢  μ₁ - μ₂ - y   ⎥
⎢                ⎥
⎣2⋅μ₁⋅y - μ₃ - x ⎦

In [12]:
roots = sympy.solve(optima_cond)
roots

In [13]:
root = roots[2]
root

### 检查

In [14]:
# 检查Hessian矩阵是否正定
f_hessian = sympy.Matrix([[f_sym.diff(x_, y_) for x_ in (x, y)] for y_ in (x, y)])
f_hessian.is_positive_semidefinite

False

In [57]:
f_max = -f_sym.subs({'x': 4./3, 'y': np.sqrt(6) / 3.})
print("最小值点：x={:.4f}, y={:.4f}, f_max={:.4f}".format(4./3, np.sqrt(6) / 3., f_max))

最小值点：x=1.3333, y=0.8165, f_max=1.0887


In [17]:
min_pt = [np.array([4./3]), np.array([np.sqrt(6) / 3.]), np.array([f_max]).astype("float")]

plt3d.figure(width=1000, height=600)
plt3d.plot_surface(xxn, fn, yyn, color='gray')
plt3d.plot_wireframe(xxn, h1n, yyn, color='green')
plt3d.plot_wireframe(xxn, h2n, yyn, color='blue')
plt3d.plot_wireframe(xxn, h3n, yyn, color='orange')
plt3d.scatter(min_pt[0], min_pt[2], min_pt[1], marker="diamond", size=5)
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

## 使用`Scipy`

In [49]:
from scipy.optimize import LinearConstraint, NonlinearConstraint
from scipy import optimize

In [50]:
linear_constraint = LinearConstraint(np.eye(2), [0, 0], [np.inf, np.inf])
nonlinear_constraint  = NonlinearConstraint(h1, 0, np.inf)

In [53]:
f = lambda X: -X[0] * X[1]
results = optimize.minimize(f, [1.3, 0.1], method='trust-constr', 
                            constraints=[nonlinear_constraint, linear_constraint])
results

 barrier_parameter: 1.2800000000000007e-06
 barrier_tolerance: 1.2800000000000007e-06
          cg_niter: 19
      cg_stop_cond: 1
            constr: [array([1.56767968e-06]), array([1.33333229, 0.81649626])]
       constr_nfev: [36, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.28801631927490234
               fun: -1.0886608278987828
              grad: array([-0.81649626, -1.33333229])
               jac: [array([[-1.        , -1.63299254]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([-2.25599735e-09,  1.38151137e-09])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 36
              nhev: 0
               nit: 19
             niter: 19
              njev: 0
        optimality: 2.2559973472467745e-09
            status: 1
           success: True
         tr_radius: 497375.62436217995
                 v: [ar

In [55]:
results.x, -results.fun

(array([1.33333229, 0.81649626]), 1.0886608278987828)